## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и наличие детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

Импортируем библиотеку Pandas, чтобы получить доступ к данным и инструментам.

In [1]:
import pandas as pd

Прочитаем файл data.csv и сохраним его в переменной df.

In [2]:
df = pd.read_csv('/datasets/data.csv')

Просмотрим данную нам таблицу, а также информацию о ней, чтобы иметь понятие о общем объеме данных.

In [3]:
print(df.head(15))

    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0    -926.185831         27               высшее             0   
6          0   -2879.202052         43               высшее             0   
7          0    -152.779569         50              СРЕДНЕЕ             1   
8          2   -6929.865299         35               ВЫСШЕЕ             0   
9          0   -2188.756445         41              среднее             1   
10         2   -4171.483647         36               высшее             0   
11         0    -792.701887         40              среднее             1   

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Нам представлена таблица, с данными клиентов банка. В ней присутствуют данные о 21525 клиентах. Сразу бросаются в глаза разные регистры, артефакты, в виде отрицательных значений, полный разброс в категориях цели займа.  

### Шаг 2. Предобработка данных

### Обработка пропусков


Определим, в каких столбцах встречаются пропущенные значения и их количество.

In [5]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Мы видим, что пропущенные значения встречаются в столбцах с количественными переменными.

Первый столбец с пропущенными значениями 'days_employed'. Он показывает нам трудовой стаж клиента. Если там пусто, можно сделать вывод, что, возможно, у клиента не имеется трудового стажа и его не стали заполнять. Может быть так, что эти пропущенные значения неслучайные, но для того, чтобы сохранялось нормальное распределение данных в этом столбце, мы не будем заполнять его нулями, а заполним медианнымми значениями.

In [6]:
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].mean())

In [7]:
df['total_income'] = df['total_income'].fillna(0)

И если в столбце 'days_employed' мы можем заполнить медианным значением всего столбца, то в 'total_income' для точности расчетов мы возьмем медианные значения в каждой группе трудоустроенности.

Определим количество клиентов в каждой категории.

In [8]:
print(df['income_type'].value_counts())

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64


Далее создаем сводную таблицу, первым столбцом которой будет категория трудоустроенности, а второй столбец - медиана столбца 'total_income'по каждой категории, использую метод pivot_table(). Мы получили готовые значения, которые нам предстоит записать в столбец главного датафрейма df 'total_income' по каждому виду занятости.

In [9]:
median_income = df.pivot_table(index=['income_type'], values=['total_income'], aggfunc='mean')
print(median_income)

                  total_income
income_type                   
безработный      131339.751676
в декрете         53829.130729
госслужащий      153679.631678
компаньон        182195.618704
пенсионер        122440.317524
предприниматель  249581.572474
сотрудник        145342.380477
студент           98201.625314


Далее заменим каждое нулевое значение на медиану по категориям.

In [10]:
numb = 0
while numb < df.shape[0]:
    df_line = df.loc[numb, 'income_type']
    df_row = df.loc[numb, 'total_income']
    if (df_line == 'безработный') & (df_row == 0):
        df.loc[numb, 'total_income'] = 131339.751676
    elif (df_line == 'в декрете') & (df_row == 0):
        df.loc[numb, 'total_income'] = 53829.130729    
    elif (df_line == 'госслужащий') & (df_row == 0):
        df.loc[numb, 'total_income'] = 170898.309923
    elif (df_line == 'компаньон') & (df_row == 0):
        df.loc[numb, 'total_income'] = 202417.461462    
    elif (df_line == 'пенсионер') & (df_row == 0):
        df.loc[numb, 'total_income'] = 137127.465690    
    elif (df_line == 'предприниматель') & (df_row == 0):
        df.loc[numb, 'total_income'] = 499163.144947    
    elif (df_line == 'сотрудник') & (df_row == 0):
        df.loc[numb, 'total_income'] = 161380.260488    
    elif (df_line == 'студент') & (df_row == 0):
        df.loc[numb, 'total_income'] = 98201.625314
    numb += 1

In [11]:
print(df.head(20))

    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0    -926.185831         27               высшее             0   
6          0   -2879.202052         43               высшее             0   
7          0    -152.779569         50              СРЕДНЕЕ             1   
8          2   -6929.865299         35               ВЫСШЕЕ             0   
9          0   -2188.756445         41              среднее             1   
10         2   -4171.483647         36               высшее             0   
11         0    -792.701887         40              среднее             1   

Проверим, работу метода, заполняющего пропущенные значения.

In [12]:
df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

И замену нуля на медианные значения:

In [13]:
df[df['total_income'] == 0].count()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Корректировка невозможных данных

Просматривая все данные мы обнаруживаем, что в столбце 'children' присутствуют данные, существование которых явно подразумевает ошибку.

In [14]:
print(df['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


А именно -1 ребенок и 20 детей. Второй вариант, вполне возможен и мог бы иметь место быть, но не в целых 76 семьях. Даже если предположить, что каждый родитель в семье берет кредит, 38 человек все равно невозможное в жизни число. Учитывая малое колличество этих значений, для того, чтобы не портить статистику мы заменим их на более реальные значения. Берем моду столбца (самое часто встречающееся значение = 0) и заменяем значения методом replace().

In [15]:
df['children'] = df['children'].replace(-1, 0)
df['children'] = df['children'].replace(20, 0)

Проверим работу замены.

In [16]:
print(df['children'].value_counts())

0    14272
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64


### Вывод

Для того, чтобы можно было проводить расчеты с таблицей, необходимо, чтобы все параметры присутствовали. Отсутствующие значения в этой таблице появились неслучайно. Так как эти параметры у клиента отсутствуют, то и в таблице их не оказалось. Но так как это количественные переменные, то для дальнейших расчетов или сравнений удобнее будет придать им их логически правильное значение.

### Замена типа данных

Мы видим, что в столбце трудового стажа содержатся артефакты. Этот столбец не нужен нам, для ответа на поставленные нам задачи, но так как он содержит данные, нам необходимо обработать их, настолько, насколько это возможно. Мы уменьшим вес данных, если дни будут представлены в целочисленном виде типа данных - integer(int).

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [18]:
df['days_employed'] = df['days_employed'].astype('int')

In [19]:
df['total_income'] = df['total_income'].astype('int')

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Так как данные выглят лаконичнее, проще и весят меньше, если имеют тип int, а не float, мы заменили их. Цифры буду восприниматься проще, если не будет десятых долей. При этом на расчетах это не сильно отразится отразится.

### Обработка дубликатов

Мы видим, что в столбце education(образование) необходимо привести значения к одному регистру, чтобы эффективно выявить все дубликаты. Воспользуемся методом str.lower()

In [21]:
df['education'] = df['education'].str.lower()
print(df.head(15))

    children  days_employed  dob_years            education  education_id  \
0          1          -8437         42               высшее             0   
1          1          -4024         36              среднее             1   
2          0          -5623         33              среднее             1   
3          3          -4124         32              среднее             1   
4          0         340266         53              среднее             1   
5          0           -926         27               высшее             0   
6          0          -2879         43               высшее             0   
7          0           -152         50              среднее             1   
8          2          -6929         35               высшее             0   
9          0          -2188         41              среднее             1   
10         2          -4171         36               высшее             0   
11         0           -792         40              среднее             1   

Воспользуемся методом df.duplicated().sum() для нахождения дубликатов.

In [22]:
df.duplicated().sum()

71

Найден 71 дубликат. Необходимо их удалить методом df.drop_duplicates() вместе с методом reset_index(), чтобы вместе с дубликатами удалились и их индексы.

In [23]:
df = df.drop_duplicates().reset_index(drop=True)

Проверим работу этого метода.

In [24]:
df.duplicated().sum()

0

### Вывод

Дубликаты - это повторяющиеся строки. Они влияют на качество расчетов и анализе как количественных, так и качественных переменных. Применив метод drop_duplicates() мы убрали их и теперь они не будут мешать в дальнейшем.

### Лемматизация

Лемматизация - это приведение слов к одной лемме. Простым языком, это замена слов в разных формах на одну общую. Это удобно для того, чтобы можно было группировать и классифицировать определенные наборы слов.

Для начала получаем лемматизатор для слов.

In [25]:
from pymystem3 import Mystem
m = Mystem()

Далее к столбцу, который нам нужно классифицировать, мы применяем метод лемматизации и на выходе получаем, какие объекты содержит каждая строка таблицы.

In [26]:
df['purpose_new'] = df['purpose'].apply(m.lemmatize)
print(df['purpose_new'])

0                             [покупка,  , жилье, \n]
1                   [приобретение,  , автомобиль, \n]
2                             [покупка,  , жилье, \n]
3                [дополнительный,  , образование, \n]
4                           [сыграть,  , свадьба, \n]
                             ...                     
21449                  [операция,  , с,  , жилье, \n]
21450               [сделка,  , с,  , автомобиль, \n]
21451                              [недвижимость, \n]
21452    [на,  , покупка,  , свой,  , автомобиль, \n]
21453             [на,  , покупка,  , автомобиль, \n]
Name: purpose_new, Length: 21454, dtype: object


Заведем свой словарь слов, какими мы хотим назвать наши категории. Жилье и недвижимость будут в разных категориях, потому, что недвижимость бывает еще и коммерческая.

In [27]:
new_dict = ['жилье', 'недвижимость', 'свадьба', 'образование', 'автомобиль']
print(new_dict)

['жилье', 'недвижимость', 'свадьба', 'образование', 'автомобиль']


Далее пишем функцию, которая будет возвращать значения из словаря, если оно будет присутствовать в наших листах в лемматизированном столбце. Потом создаем новый столб, в котором эти значения будут записаны. Таким образом у нас будет всего 5 целей займа.

In [28]:
def find_purpose(purpose_list):
    for purpose in new_dict:
        if purpose in purpose_list:
            return purpose

df['purpose_new'] = df['purpose_new'].apply(find_purpose)
print(df.head(15))

    children  days_employed  dob_years            education  education_id  \
0          1          -8437         42               высшее             0   
1          1          -4024         36              среднее             1   
2          0          -5623         33              среднее             1   
3          3          -4124         32              среднее             1   
4          0         340266         53              среднее             1   
5          0           -926         27               высшее             0   
6          0          -2879         43               высшее             0   
7          0           -152         50              среднее             1   
8          2          -6929         35               высшее             0   
9          0          -2188         41              среднее             1   
10         2          -4171         36               высшее             0   
11         0           -792         40              среднее             1   

Проверим работу функции и все ли заменились элементы в строках. Методом value_counts() определим какие значения присутствуют в столбце 'purpose_new' и сколько существует строк с определенной категорией.

In [29]:
print(df['purpose_new'].value_counts())

недвижимость    6351
жилье           4460
автомобиль      4306
образование     4013
свадьба         2324
Name: purpose_new, dtype: int64


### Вывод

В начале мы получили столбец, с разными целями, не принадлежащими к определенным категориям. Они все типичны, но записаны по- разному, поэтому мы не могли определить их количество. После применения функции и после создания нового столбца мы можем увидеть какие цели займа бывают и сколько человек указали их своей целью.

### Категоризация данных

В столбце ежемесячный доход у нас очень много различных данных и исходя из этого у нас не получится конкретных выводов. Поэтому тут мы представим данные под категориями 'низкий доход', 'средний доход', 'высокий доход'. Значения уровней дохода зададим по распределению с помощью метода describe(), где <25% - низкий доход, 25% - 50% - ниже среднего, 50-75% - средний, >75% высокий.

In [30]:
df['total_income'].describe().astype('int')

count      21454
mean      167431
std        98060
min        20667
25%       107623
50%       151887
75%       202417
max      2265604
Name: total_income, dtype: int64

In [31]:
def cat_income(row):
    if row <= 107623:
        return 'низкий доход'
    elif (row > 107623) & (row < 151887):
        return 'доход ниже среднего'
    elif (row > 151887) & (row < 202417):
        return 'средний доход'
    else:
        return 'высокий доход'

In [32]:
df['categody_income'] = df['total_income'].apply(cat_income)
print(df.head(15))

    children  days_employed  dob_years            education  education_id  \
0          1          -8437         42               высшее             0   
1          1          -4024         36              среднее             1   
2          0          -5623         33              среднее             1   
3          3          -4124         32              среднее             1   
4          0         340266         53              среднее             1   
5          0           -926         27               высшее             0   
6          0          -2879         43               высшее             0   
7          0           -152         50              среднее             1   
8          2          -6929         35               высшее             0   
9          0          -2188         41              среднее             1   
10         2          -4171         36               высшее             0   
11         0           -792         40              среднее             1   

In [33]:
print(df['categody_income'].value_counts())

высокий доход          5410
низкий доход           5364
доход ниже среднего    5363
средний доход          5317
Name: categody_income, dtype: int64


### Вывод

Мы распределили ежемесячный доход по трем категориям, чтобы было удобнее считать зависимость. Теперь мы четко видим к какой категории относится каждый заемщик.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для того, чтобы определить зависимость, мы применим группировку к столбцу 'children' и на вывод поставить среднее значение столбца 'debt' по каждой категории. Так как в столбце 'debt' у нас 1 - задолженность, то на выходе мы получаем вероятность невозврата займа. 

Этот метод мы будем применять, для ответа на все вопросы.

In [34]:
print(df.groupby('children').agg({'debt':'mean'}))

              debt
children          
0         0.075419
1         0.092346
2         0.094542
3         0.081818
4         0.097561
5         0.000000


### Вывод

Мы можем увидеть, что категория людей без детей чаще всего берут кредиты и при это чаще возвращают. И при этом люди уже хотя бы с одним ребенком реже берут, и при этом чаще задерживают выплаты. И чем больше детей, тем реже люди берут кредиты, мы можем сказать, что наличие хотя- бы одного ребенка, может быть причиной, для задержки выплаты кредита в срок.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [35]:
print(df.groupby('family_status').agg({'debt':'mean'}))

                           debt
family_status                  
Не женат / не замужем  0.097509
в разводе              0.071130
вдовец / вдова         0.065693
гражданский брак       0.093471
женат / замужем        0.075452


### Вывод

Мы можем увидеть, что если человек не в семье и не был в ней, чаще не возвращает займ в срок, чем человек, обременненый обязательствами. Скорее всего у семейных людей уровень ответственности выше, потому, они не могут себе позволить быть должниками.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [36]:
print(df.groupby('categody_income').agg({'debt':'mean'}))

                         debt
categody_income              
высокий доход        0.069316
доход ниже среднего  0.088570
низкий доход         0.079605
средний доход        0.087267


### Вывод

Мы видим, что человек с высоким доходом чаще возвращает кредит в срок. Разница между средним и низким доходом не однозначная. Возврат зависит от уровня дохода, чем выше доход, тем чаще возврат.

- Как разные цели кредита влияют на его возврат в срок?

In [37]:
print(df.groupby('purpose_new').agg({'debt':'mean'}))

                  debt
purpose_new           
автомобиль    0.093590
жилье         0.069058
недвижимость  0.074634
образование   0.092200
свадьба       0.080034


### Вывод

Мы видим, что вероятность невозврата гораздо выше, чем меньше денег нужно на цель займа. Больший невозврат в срок имеют категории 'жилье' и 'образование', и это, по идее, должно быть в разы меньше, чем недвижимость.

### Шаг 4. Общий вывод

Самая большая вероятность по невозврату оказалась у людей с 4 детьми(0.097561), а самая маленькая у вдов/вдовцов(0.065693). Самая явная зависимость видна в категории уровня дохода. И это понятно, чем больше у человека доход, тем больше вероятность, что он сможет отдать займ. Семейный статус имеет очень большой разброс и не покажет вероятность также эффективно.